<a href="https://colab.research.google.com/github/hailthedawn/BISFinalProject/blob/main/BetterModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pathlib2

In [ ]:
import tensorflow as tf
import glob
import tensorflow_datasets as tfds
from tensorflow.io import FixedLenFeature, parse_single_example
#from librosa.core.time_frequency import mel_frequencies
from pathlib2 import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal


### Download or load dataset


In [ ]:
DATA_DIR = Path("data").resolve()
DATA_DIR

PosixPath('/content/data')

In [ ]:
ds_train, ds_test = tfds.load(
    name="nsynth", split=["train", "test"], data_dir=DATA_DIR
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/1069 [00:00<?, ? file/s]


Dataset nsynth downloaded and prepared to /content/data/nsynth/full/2.3.3. Subsequent calls will reuse this data.


### Produce the datasets from tfrecords

In [ ]:
training_tfrecords = [str(i) for i in list((DATA_DIR / "nsynth").glob('**/*train.tfrecord*'))]
raw_dataset = tf.data.TFRecordDataset(training_tfrecords)

val_tfrecords = [str(i) for i in list((DATA_DIR / "nsynth").glob('**/*valid.tfrecord*'))]
val_dataset = tf.data.TFRecordDataset(val_tfrecords)

test_tfrecords = [str(i) for i in list((DATA_DIR / "nsynth").glob('**/*test.tfrecord*'))]
test_dataset = tf.data.TFRecordDataset(test_tfrecords)

**Function to label classes from ints**

In [ ]:

def label_instrument(num):
  if num == 0:
    return "bass"
  if num == 1:
    return "brass"
  if num == 2:
    return "flute"
  if num == 3:
    return "guitar"
  if num == 4:
    return "keyboard"
  if num == 5:
    return "mallet"
  if num == 6:
    return "organ"
  if num == 7:
    return "reed"
  if num == 8:
    return "string"
  if num == 9:
    return "vocal"
  else:
    return "invalid class"

BETTER NETWORK

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv1D, Activation, Dropout, MaxPooling1D, Flatten, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from scipy import signal

**READ IN TRAINING DATA**

In [ ]:
#raw_dataset = raw_dataset
X=[]
y=[]
counts = dict()
#names=[]
counter=0
with tf.device('/device:GPU:0'):
  for raw_record in raw_dataset.take(50000):
     example = tf.train.Example()
     example.ParseFromString(raw_record.numpy())
     #print(example)
     temp=example.features.feature["instrument/family"].int64_list.value[0]
     #print(temp)
     if(temp!=9):          
       if(counts.get(temp)==None or counts.get(temp)<4000):
          X.append(example.features.feature["audio"].float_list.value[:16000])
          y.append(example.features.feature["instrument/family"].int64_list.value[0])
          
          counts[temp] = counts.get(temp, 0) + 1
       if counts.get(temp)>4000:
          break
              


yClass=[]

for i in range(0,len(y)):
    temp=[0] * 11
    temp[y[i]]=1
    yClass.append(temp)
yTrain=np.array(yClass)
counter=0

XTrain= np.array(X)


(9886, 160, 100)

**NEXT READ IN VAL DATA**

In [ ]:
X=[]
y=[]
counts = dict()
#names=[]
counter=0
for raw_record in val_dataset.take(3500):
     example = tf.train.Example()
     example.ParseFromString(raw_record.numpy())
     #print(example)
     temp=example.features.feature["instrument/family"].int64_list.value[0]
     #print(temp)
     #print(temp)
     if(temp!=9):          
       if(counts.get(temp)==None or counts.get(temp)<1000):
          X.append([example.features.feature["audio"].float_list.value[:16000]])
          y.append(example.features.feature["instrument/family"].int64_list.value[0])
          
          counts[temp] = counts.get(temp, 0) + 1
         
#print(y)
yClass=[]
#counts=None
for i in range(0,len(y)):
    temp=[0] * 11
    temp[y[i]]=1
    yClass.append(temp)
yVal=np.array(yClass)
#yVal=np.array(y)
Xnew=[]
counter=0
for i in  X:
    #counter=1+counter
    Xnew.append(i)
XVal= np.array(Xnew)

In [ ]:
XTrain = XTrain.reshape(XTrain.shape[0], 160, 100)
XVal = XVal.reshape(XVal.shape[0], 160, 100)


**1D Convolutional Model**

**Train Function**

**LSTM model**

In [ ]:

model = Sequential()
model.add(LSTM(128, input_shape=( 160, 100), recurrent_dropout=0.2))
# model.add(LSTM(256, return_sequences=True))
# model.add(LSTM(128, return_sequences=False))
model.add(Dense( 256, activation='relu'))
model.add(Dense(11, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
  history_lstm = model.fit(XTrain, yTrain, validation_data=(XVal,yVal),epochs=20, batch_size=2000)

In [ ]:
model = Sequential()
forward_layer = LSTM(100, return_sequences=True, recurrent_dropout=0.2)
backward_layer = LSTM(100, activation='relu', return_sequences=True,
                       go_backwards=True, recurrent_dropout=0.2)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                         input_shape=(160, 100)))
model.add(Flatten())
model.add(Dense( 512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense( 256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense( 128, activation='relu'))
model.add(Dense(11, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# with tf.device('/device:GPU:0'):
history_lstm = model.fit(XTrain, yTrain, validation_data=(XVal,yVal),epochs=30, batch_size=1000)

Epoch 1/30
32/32 [==============================] - 39s 993ms/step - loss: 2.0435 - accuracy: 0.2487 - val_loss: 1.6671 - val_accuracy: 0.3277
Epoch 2/30
32/32 [==============================] - 31s 985ms/step - loss: 1.6938 - accuracy: 0.3789 - val_loss: 1.5699 - val_accuracy: 0.3909
Epoch 3/30
32/32 [==============================] - 31s 981ms/step - loss: 1.5020 - accuracy: 0.4456 - val_loss: 1.5887 - val_accuracy: 0.3866
Epoch 4/30
32/32 [==============================] - 31s 971ms/step - loss: 1.3566 - accuracy: 0.5009 - val_loss: 1.5671 - val_accuracy: 0.4177
Epoch 5/30
32/32 [==============================] - 31s 980ms/step - loss: 1.2248 - accuracy: 0.5537 - val_loss: 1.6320 - val_accuracy: 0.4263
Epoch 6/30
32/32 [==============================] - 31s 984ms/step - loss: 1.0895 - accuracy: 0.6057 - val_loss: 1.6174 - val_accuracy: 0.4634
Epoch 7/30
32/32 [==============================] - 31s 982ms/step - loss: 0.9645 - accuracy: 0.6520 - val_loss: 1.6366 - val_accuracy: 0.4700

TESTING

**LOAD IN Xtest and Ytest**
 - Size of Test Dataset can be changed with the arguement to take()

In [ ]:
X=[]
y=[]
counts = dict()
#names=[]
counter=0
for raw_record in test_dataset:
     example = tf.train.Example()
     example.ParseFromString(raw_record.numpy())

     temp=example.features.feature["instrument/family"].int64_list.value[0]   
     if(temp!=9):
 
          if(counts.get(temp)==None or counts.get(temp)<1000):
            X.append(example.features.feature["audio"].float_list.value[:16000])
            y.append(example.features.feature["instrument/family"].int64_list.value[0])
          
            counts[temp] = counts.get(temp, 0) + 1

yClass=[]

for i in range(0,len(y)):
    temp=[0] * 11
    temp[y[i]]=1
    yClass.append(temp)
yTest=np.array(yClass)
#yVal=np.array(y)
Xnew=[]
counter=0

XTest= np.array(X)

In [ ]:
model.save('BiLSTM-Model')

INFO:tensorflow:Assets written to: BiLSTM-Model/assets


INFO:tensorflow:Assets written to: BiLSTM-Model/assets


**LOAD IN AND TEST THE MODEL**
Only the last cell creating XTest and YTest needs to be run.

Pass in path to the Better-Model File below

In [ ]:
model=tf.keras.models.load_model('drive/MyDrive/Better-Model')

**Evaluate accuracy on test data**

In [ ]:
XTest = XTest.reshape(XTest.shape[0], 160, 100)
_, accuracy = model.evaluate(XTest, yTest)

128/128 [==============================] - 12s 85ms/step - loss: 3.1978 - accuracy: 0.5066
